In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break
    
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
if os.path.exists("model.pth"):
    model.load_state_dict(torch.load("model.pth"))
    print("Loaded model from model.pth")
else:
    print("No model found, creating new model")
print(model)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x.to(device))
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

In [ ]:
import os
import torch
from torch import nn
import sqlite3
import sys
import traceback
import numpy as np


#Connecting to the SQLite database
dbPath = r'Data\datasetNY.db'
dbConnection = sqlite3.connect(dbPath)

dbData = []

#Retrieve the data from the database
try:
    dbData = dbConnection.execute('SELECT * FROM Motor_Vehicle_Collisions_Crashes').fetchall()
except:
    print('Error retrieving the data from the database')
else:
    print('Successfully retrieved the data from the databse')

first = int(float(dbData[1][2]))

n_input = first
n_hidden = 15
n_out = 1
batch_size = 100
learning_rate = 0.01

data_x = torch.randn(batch_size, n_input)
data_y = (torch.rand(size=(batch_size, 1)) < 0.5).float()

print(data_x.size())
print(data_y.size())

model = nn.Sequential(nn.Linear(n_input, n_hidden),
                      nn.ReLU(),
                      nn.Linear(n_hidden, n_out),
                      nn.Sigmoid())
print(model)

loss_function = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

losses = []
for epoch in range(5000):
    pred_y = model(data_x)
    loss = loss_function(pred_y, data_y)
    losses.append(loss.item())

    model.zero_grad()
    loss.backward()

    optimizer.step()
    
import matplotlib.pyplot as plt
plt.plot(losses)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.title("Learning rate %f"%(learning_rate))
plt.show()

dbConnection.close

In [31]:
import os
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import datetime
import sqlite3
import sys
import traceback
import numpy as np
import Data.database_handler as dbHandler

#Connecting to the SQLite database
dbPath = r'Data\datasetNY.db'

class AccidentDataset(Dataset):
    def __init__(self, db_path, transform=None):
        self.coordinates = dbHandler.get_all_data_datetime_converted(db_path)
        self.coordinates = pd.DataFrame(self.coordinates, columns=['datetime', 'latitude', 'longitude'])
        self.transform = transform

    def __len__(self):
        return len(self.coordinates)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        datetime = self.coordinates.iloc[idx, 0:1]
        datetime = np.array([datetime]).squeeze()
        datetime = datetime.astype('float')
        datetime = torch.from_numpy(datetime).float()

        coordinates = self.coordinates.iloc[idx, 1:3]
        coordinates = np.array([coordinates]).squeeze()
        coordinates = coordinates.astype('float')
        coordinates = torch.from_numpy(coordinates).float()
        sample = {'datetime' : datetime, 'coordinates': coordinates}

        if self.transform:
            sample = self.transform(sample)

        return sample


#     #Making constructor
#     def __init__(self, dbData, root_dir, transform_none):
#         self.coordinates = dbData
#         self.root_dir = root_dir
#         self.transform = transform_none
# https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
# load data from database and split into train and test


accident_dataset = AccidentDataset(dbPath)

print(accident_dataset[0])
print(accident_dataset[0]['coordinates'][0])
print(accident_dataset[0]['datetime'])

#Create new array with 60% of the data
train_size = int(0.01 * len(accident_dataset))
test_size = len(accident_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(accident_dataset, [train_size, test_size])

print(len(train_dataset))
print(len(test_dataset))
print(len(accident_dataset))

#Create dataloader
train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=False)

#Creating Network class
class Network(nn.Module):

    #Making constructor
    def __init__(self, input_size, number_of_classes):
        super(Network, self).__init__()
        #Creating fully connected layers
        self.fclayer1 = nn.Linear(input_size, 50)
        self.fclayer2 = nn.Linear(50, number_of_classes)
    
    #Creating forward method to do forward propagation
    def forward(self, input):
        input = nn.functional.relu(self.fclayer1(input))
        input = self.fclayer2(input)
        return input
    

#Get cpu or gpu device for training.
device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f'Using {device} device')

model = Network(10, 2).to(device)
if os.path.exists("model.pth"):
    model.load_state_dict(torch.load("model.pth"))
    print("Loaded model from model.pth")
else:
    print("No model found, creating new model")
print(model)

loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    print(size)
    model.train()
    print("Training model")
    for i, batch in enumerate(dataloader):
        X, y = batch['datetime'], batch['coordinates']
        print(X, y)
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    print("Finished training model")
            
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!") 

torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

[1631345700.0, '40.667202', '-73.8665']
{'datetime': tensor(1.6313e+09), 'coordinates': tensor([ 40.6672, -73.8665])}
tensor(40.6672)
tensor(1.6313e+09)
17415
1724172
1741587
Using cuda device
No model found, creating new model
Network(
  (fclayer1): Linear(in_features=10, out_features=50, bias=True)
  (fclayer2): Linear(in_features=50, out_features=2, bias=True)
)
Epoch 1
-------------------------------
17415
Training model
tensor([1.4393e+09, 1.5375e+09, 1.4409e+09, 1.6534e+09, 1.3740e+09, 1.4931e+09,
        1.6136e+09, 1.5694e+09, 1.4135e+09, 1.4345e+09]) tensor([[ 40.6960, -73.9435],
        [ 40.8538, -73.9073],
        [ 40.5684, -74.1413],
        [ 40.6642, -73.9455],
        [ 40.7675, -73.8819],
        [ 40.7030, -73.9443],
        [ 40.7162, -74.0026],
        [ 40.6573, -73.9741],
        [ 40.7453, -73.9985],
        [ 40.8283, -73.8829]])


d:\Programmer\Anaconda\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([10, 2])) that is different to the input size (torch.Size([2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


TypeError: unsupported operand type(s) for %: 'dict' and 'int'